In [ ]:
from qutip import *
import matplotlib.pyplot as plt
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize']  = 18
plt.rcParams['axes.titlesize']  = 16
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 16

folder_name = '../Data/'

In [ ]:
N          = 40

phi_vec    = np.linspace(0, 1 * np.pi, 60)
theta_vec  = np.linspace(0, 1 * np.pi, 60)

CFI_angles = []

for phi in phi_vec:

    div           = 1000

    #From Figure 2 - BTC file:
    QFI_vec       = np.load(folder_name + "QFI_along_w0_N="+str(N)+".npy")
    w0_vec        = np.load(folder_name + "w0_N="+str(N)+".npy")

    where_max_QFI = w0_vec[ np.argmax(QFI_vec) ]

    CFI_theta     = []

    for theta in theta_vec:

        J_x          = jmat( N/2, 'x')
        J_y          = jmat( N/2, 'y')
        J_z          = jmat( N/2, 'z') 

        J_minus      = jmat( N/2, '-')

        J_O          = np.sin(theta) * np.cos(phi) * J_x + \
                       np.sin(theta) * np.sin(phi) * J_y + \
                       np.cos(theta) * J_z
                
        eig_J_O      = J_O.eigenstates()

        #eig_J_O[1][s]

        kappa        = 1
        c_ops        = [np.sqrt( kappa / (N/2) ) * J_minus]

        w0           = where_max_QFI

        tol          = 1e-12

        ########### no increment
        H            = w0 * J_x
        rho          = steadystate(H, c_ops, method = 'direct', use_rcm = True)

        delta_w0     = w0 / div

        ########### plus increment
        H_plus       = (w0 + delta_w0) * J_x
        rho_plus     = steadystate(H_plus , c_ops, method = 'direct', use_rcm = True)

        ########### minus increment
        H_minus      = (w0 - delta_w0) * J_x
        rho_minus    = steadystate(H_minus, c_ops, method = 'direct', use_rcm = True)

        ########### 2 plus increment
        H_2plus      = (w0 + 2 * delta_w0) * J_x
        rho_2plus    = steadystate(H_2plus , c_ops, method = 'direct', use_rcm = True)

        ########### 2 minus increment
        H_2minus     = (w0 - 2 * delta_w0) * J_x
        rho_2minus   = steadystate(H_2minus, c_ops, method = 'direct', use_rcm = True)

        cfi_s = 0

        for s in range( len(eig_J_O[0]) ) :
            diff        = (-rho_2plus + 8*rho_plus - 8*rho_minus + rho_2minus) / (12 * delta_w0)
            prob_s      = (eig_J_O[1][s].dag() * rho * eig_J_O[1][s])[0,0]
            diff_prob_s = (eig_J_O[1][s].dag() * diff * eig_J_O[1][s])[0,0]

            if prob_s >= tol:
                cfi_s = cfi_s + (1 / prob_s) * (diff_prob_s)**2

        CFI_theta.append( np.real(cfi_s) )
        
    CFI_angles.append( CFI_theta )